In [3]:
print(vocab._count)

Counter({'\n': 999, 'the': 457, '': 336, 'i': 253, 'a': 236, 'to': 229, 'you': 203, 'and': 196, 'it': 146, 'is': 125, "i'm": 108, 'my': 95, 'of': 90, 'in': 84, 'this': 75, 'your': 74, 'like': 71, "it's": 66, 'so': 65, 'for': 64, 'me': 64, 'that': 62, 'up': 61, 'when': 56, 'on': 54, 'be': 51, 'get': 51, 'but': 47, 'rhyme': 44, 'then': 42, 'can': 39, 'still': 38, 'mic': 38, 'no': 37, 'if': 36, 'from': 35, 'euphoria': 34, 'with': 34, 'or': 33, 'as': 33, 'all': 32, "don't": 32, 'one': 31, 'what': 30, 'just': 30, 'time': 29, 'was': 28, 'by': 27, 'out': 27, 'they': 27, 'make': 27, 'we': 26, 'ya': 26, 'rakim': 25, 'off': 25, 'see': 25, 'got': 25, "can't": 23, "you're": 23, 'know': 22, 'keep': 22, 'microphone': 22, 'cause': 21, "ain't": 20, "i'll": 20, 'into': 19, 'take': 19, 'not': 19, 'let': 19, 'rhymes': 18, 'first': 17, 'world': 17, 'new': 17, 'go': 17, "that's": 17, 'here': 17, 'have': 17, 'been': 16, 'put': 16, "'em": 16, 'now': 16, 'cuz': 16, 'at': 15, 'start': 15, 'verse': 15, "'cause"

In [1]:
import tools.processing as pre
import tools.architectures as nn

text = pre.get_text("data/cleaned-rap-lyrics/cleanrakim_.txt")

vocab = pre.Vocabulary(text)

NUM_LAYERS = 3
HIDDEN_LAYER_SIZE = 256
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256


data, labels = vocab.making_one_hot(text, TIMESTEPS)


rnn = nn.MultiLayerRNN(name = "multi-rakim")
rnn.build(NUM_LAYERS, HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1)



Epoch 1/20
Loss:    	 7.509294509887695
Accuracy:	 0.01032204832881689
Seed:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to 
Result:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to try b alphabetically in by stage a my  home take is we pull the is which smooth so i i he to you're but to so 
 get than and rakim middle this 
  topic melody you're  
 true  of 
 
  who that into  night my me had  by you ask when from originator a follow with the beat missus by  to e canibus boric bodega   and  times for finish from in  here at can't better  trying  make don't these it stand your at a five  if me best a first let  jadakiss around weak i you my if drop if hope  i in slept go can't in get mind put  genuine remain  you and here droppin you effect 
   break i the i'll but and the boric stand 
 so and up 
 this be 
 a mind 
 my together is  like i'll still a  it me this playing wo

KeyboardInterrupt: 